In [21]:
# Retroactively adding these asa they're needed for exporting images
# Need these for exporting images
%pip install -U kaleido
%pip install -U plotly
!plotly_get_chrome
import kaleido
import plotly.io as pio

# Import needed libraires
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


Plotly will install a copy of Google Chrome to be used for generating static images of plots.
Chrome will be installed at: None
Do you want to proceed? [y/n] y
Installing Chrome for Plotly...
Chrome installed successfully.
The Chrome executable is now located at: /usr/local/lib/python3.12/dist-packages/choreographer/cli/browser_exe/chrome-linux64/chrome


In [22]:
# Import from GitHub repository
stuperf = pd.read_csv('https://raw.githubusercontent.com/RoobyDoobyDoo/CS5530-Assign1/refs/heads/main/StudentsPerformance/StudentsPerformance.csv')
stuperf

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


# Check and clean the data

In [23]:
# SStart with checking for missig values
stuperf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [24]:
# No nulls, nice
stuperf.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [25]:
# Looks realistic so far, one last check
stuperf.duplicated().sum()

np.int64(0)

In [26]:
# Looks like I won't have to filter out any samples here
# I'll get rid of any columns that are unecessary for this assignment first
stuperf.drop(columns=['race/ethnicity', 'parental level of education'], inplace=True)
stuperf

,gender,lunch,test preparation course,math score,reading score,writing score
0,female,standard,none,72,72,74
1,female,standard,completed,69,90,88
2,female,standard,none,90,95,93
3,male,free/reduced,none,47,57,44
4,male,standard,none,76,78,75
...,...,...,...,...,...,...
995,female,standard,completed,88,99,95
996,male,free/reduced,none,62,55,55
997,female,free/reduced,completed,59,71,65
998,female,standard,completed,68,78,77


In [27]:
# Let's see what we'll be working with here
stuperf.nunique()

,0
gender,2
lunch,2
test preparation course,2
math score,81
reading score,72
writing score,77


In [28]:
stuperf.dtypes

,0
gender,object
lunch,object
test preparation course,object
math score,int64
reading score,int64
writing score,int64


## I did a bit of graphing for the first question before I realized you'd already specified how you wanted us to show and compare the data. It's not relevant for the answers, but I'll leave it in to show that I was working on learning Plotly.
## The actual responces are further below.

In [29]:
hist = make_subplots(rows=2, cols=2, subplot_titles=('Male Math Scores', 'Female Math Scores', 'Male Reading Scores', 'Female Reading Scores'))

hist.add_trace(go.Histogram(x=stuperf[stuperf['gender'] == 'male']['math score'], name='Male'), row=1, col=1)
hist.add_trace(go.Histogram(x=stuperf[stuperf['gender'] == 'female']['math score'], name='Female'), row=1, col=2)
hist.add_trace(go.Histogram(x=stuperf[stuperf['gender'] == 'male']['reading score'], name='Male'), row=2, col=1)
hist.add_trace(go.Histogram(x=stuperf[stuperf['gender'] == 'female']['reading score'], name='Female'), row=2, col=2)

hist.update_layout(title=dict(text='<b>Male and Female Score Distributions in Math and Reading</b>',
                             font=dict(size=24)
                             ),
                  showlegend=False,
                  )

hist

In [30]:
# Get mean score data for relevant columns
meanmath = stuperf.groupby('gender')['math score'].mean()
meanread = stuperf.groupby('gender')['reading score'].mean()

# Couldn't simply just merge the two simple datasets
# Had to reorient the data a little to make it easier to use with Plotly
mean = pd.concat([meanmath, meanread], axis=1).reset_index()
mean = mean.melt(id_vars='gender', var_name='subject', value_name='mean score')

bar = px.bar(mean, x='subject', y='mean score',
             color='gender', barmode='group',
             labels=dict(x='Subject', y='Mean Score', color='Gender'),
             title='Comparing Mean Scores between Male and Female Students'
             )

bar.update_layout(title=dict(font=dict(size=24)))

bar

# V1 -- Gender Boxplots (Math vs Reading)

In [31]:
# Melt the dataframe to have 'variable' and 'value' columns for eassy plotting
stuperf_melted = stuperf.melt(id_vars=['gender'],
                              value_vars=['math score', 'reading score'],
                              var_name='subject',
                              value_name='score'
                              )

# Blot Pox
genscore = px.box(stuperf_melted,
                  x="subject", y="score", color="gender",
                  width=800, height=600,
                  title="Score Distribution by Gender for Math and Reading"
                  )

genscore.update_layout(title=dict(font=dict(size=24)))

genscore

## Question: Are there gender differences in math or reading?
There are minor differences in the scores and their distribution between genders here. It's not very significant but male students seemed to perform better on average than female students in math and vice versa in reading.

There is a bigger gap between the genders in reading scores, with female students performing closer to almost a letter grade better, whereas male students were closer to half a grade level higher in math scores. The scores for female students also appear to be more spread out or distributed more widely than the male students' scores. Since the max score is 100, we can't tell if there would've been a greater spread for the higher scores. We might be seeing a smaller difference than if the tests were set to have no upper limit. That may allow some of the best test-takers to stand out further ahead and change the distribution accordingly to be more like a standard deviation bell curve.


# V2 -- TestPrep Impact on Math

In [32]:
# Since there's only one continuous axis to look at, I think the box plot is the best option
testprep = px.box(stuperf,
                  x='test preparation course', y='math score',
                  width=800, height=600,
                  title='Math score distribution vs Studying'
                  )

testprep.update_layout(title=dict(font=dict(size=24)))

testprep

## Question: Do students who completed test prep score higher in math?
Students who completed the test prep course did do slightly better than students who didn't take any test prep at all.

Studying did a great job in shrinking the distribution, significantly fewer students had lower scores. It appears that test prep is most helpful to those who are struggling with the course information, at least in the case of math. It didn't appear to raise top performers' scores by much, but that may be more due to the 100-score limit than the students' ability (like what I mentioned in the previous question). This didn’t really change the shape of the distribution much; it seems to have shrunk towards higher scores. I would think that maybe increasing the upper limit may show us if test prep mainly skews the lower performing students higher or if all students perform relatively better. Maybe adding extra credit would be a good way to stretch the upper limits and look there for findings on if/how it changes the distribution.

# V3 -- Lunch Type and Average Performance

In [33]:
# Calculate the average scores by lunch type for math, reading, and writing
meanlunch = stuperf.groupby('lunch')[['math score', 'reading score', 'writing score']].mean().reset_index()

# Melt for plotly
meanmelt = meanlunch.melt(id_vars='lunch',
                          var_name='subject',
                          value_name='average score'
                          )

# Create the grouped bar chart
lunchbar = px.bar(meanmelt,
                  x='subject', y='average score', color='lunch', barmode='group',
                  width=800, height=600,
                  title='Mean Scores for Math, Reading, and Writing by Lunch Type')

lunchbar.update_layout(title=dict(font=dict(size=24)))

lunchbar

## Question: Does lunch type (standard vs free/reduced) relate to outcomes?
Yes, we can see that average scores are higher for students who had free/reduced lunch versus students who had to pay full.

The (average) increase is a whole letter grade for reading and writing, and just barely 2 letter grades for math. This means that students who are given easier access to lunches perform better on tests, whether this is purely in test-taking or overall academic performance isn’t directly displayed here. However, we can still interpret it as students given more affordable lunches are able to focus more on studies and/or retain the relevant course information for test-taking. Further interpreting that easier access to food lets students have more nutrition from lunch, which can lead to better health and minds, and thus better academic performance and scores. We can draw upon this further if we consider that some students may not be getting lunch at all if they skip lunch when having to pay for it, although we don't the data that explicitly states this.

# V4 -- Subject Correlations

In [34]:
# Gotta get correlation values for the relevant columns first
corrscore = stuperf[['math score', 'reading score', 'writing score']].corr()

# Then plot, we need to add the annotations manually
heatmap = px.imshow(corrscore,
                    text_auto=True,
                    width=800, height=600,
                    color_continuous_scale='RdBu',
                    title='Correlation Heatmap of Math, Reading, and Writing Scores')

heatmap.update_layout(title_font=dict(size=24),)

heatmap

## Question: How strongly do the three subjects move together?
It seems that reading and writing scores correlate quite well with one another, while math doesn't seem to have as strong of a correlation with either reading or writing.

This makes sense when considering reading and writing are both language/communication-based subjects so we should expect them to be closely tied together; students who perform well in reading should also have a solid understanding in writing and vice versa. The similarity lines up as math has almost the same correlation with reading as writing. While math, being a more logic-based subject, didn't have as high of a correlation with the other two subjects, it still has a pretty strong relationship. It's likely that students who are performing well in one subject are also likely to perform well in school all together. Studious students often study harder on all their schoolwork and struggling (or slacking) students are often having trouble with all the work cumulatively as each subject’s coursework may keep piling up on one another.



# V5 -- Math vs Reading w/ Trend lines by Test Prep

In [35]:
# Count the total amount of students who did and didn't take test prep
prepcount = stuperf['test preparation course'].value_counts()

# Create a scatter plot of reading vs math scores, colored by test preparation course
# Using a simple least squares method for line of best fit
scatmr = px.scatter(stuperf,
                    x='reading score', y='math score', color='test preparation course',
                    width=800, height=600,
                    title='Math vs Reading Scores by Test Prep',
                    trendline='ols')

# Have to go in and manually add the counts for each label
for i, data in enumerate(scatmr.data):
    if data.name in prepcount:
        data.name = f'{data.name} ({prepcount[data.name]})'

# I wanted the legend moved since it took up so much space
scatmr.update_layout(title_font=dict(size=24),
                     legend=dict(yanchor="top", y=0.99,
                                 xanchor="left", x=0.01
                                 )
                     )

scatmr

## Question: How strongly are math and reading scores associated, and do students who completed the test‑preparation course have a different slope in the math–reading relationship than those who did not?
The math and reading scores appear to be pretty closely associated, as I mentioned in the previous question about students' tendencies in performance across the board. The relationship between them appears to be near identical when observing the trend lines.

It even appears that the trend line is ever so slightly (and insignificantly) lower with the test prep completion group. Although I believe that is because those students have fewer lower scores overall than those who didn't take the test prep, the trend line itself is shorter, starting at higher scores. The same appears true here as it did when looking at just the math score distribution vs test prep we did above: that test prep decreases the chances of getting a lower score for those who were already struggling or in greater need of assistance. I think this is another spot where having the 100-point hard barrier may be a limitation in seeing what the already higher-performing students can actually achieve when given more assistance. Although the more important takeaway here should be that studying will be beneficial to almost all students, especially those who are having a harder time with the course/material.

# Exporting
I tried for hours to get plotly to export properly. First, it needed another library called kaleido. Sure, I installed it. Then I had to troubleshoot why kaleido didn't seem to be detected at all, and eventually I figured out that this version of plotly wasn't compatible. Alright, I'll update plotly and run everything again, now kaleido needs Google Chrome installed. Why? Whatever, I installed it and now it finally works.

But now, exporting at the current resolution makes the image tiny and almost unreadable. Plus the default DPI for these PNGs is 96. I don't know how to modify that here, the documentation and online forums all say that DPI isn't an option with Plotly and Kaleido, and I have to do it via pixel count, but keeping the 800x600 px value leads us back to the same problem. I'm not sure if I'm interpreting this part of the assignment completely wrong, but I'm just gonna settle with scaling the image up by 3 and call it there for my sanity.

In [40]:
# I forgot to export any of these

# Sadly, as a PNG, we'll lose the interactiveness
pio.write_images([genscore, testprep, lunchbar, heatmap, scatmr],
    file=['V1. Gender vs Score.png',
          'V2.Math Score vs Test Prep.png',
          'V3. Lunch Type vs Mean Score.png',
          'V4. Score Correlation.png',
          'V5. Math & Reading vs Test Prep.png'],
                 scale=3
                 )